# Missing Data: Exploratory Data Analysis

When it comes to handling missing data, the effectiveness of our approach depends on the missing mechanism. This is where Exploratory Data Analysis (EDA) steps in. EDA helps us understand the missing mechanism, empowering us to make informed decisions on how to deal with those pesky null values.

We'll start from scratch, using the titanic dataset. Our initial goal is to prepare the dataset for analysis, then we can go ahead and explore the missing mechanism.

To familiarize yourself with the data, you can have a look at the dictionary below provided by kaggle, which gives a brief description for each variable.

| Variable  | Definition               | Key                                            |
|-----------|--------------------------|------------------------------------------------|
| survived  | Survival                 | 0 = No, 1 = Yes                               |
| pclass    | Ticket class             | 1 = 1st, 2 = 2nd, 3 = 3rd                     |
| sex       | Sex                      |                                                |
| Age       | Age in years             |                                                |
| sibsp     | # of siblings/spouses    |                                                |
|           | aboard the Titanic       |                                                |
| parch     | # of parents/children    |                                                |
|           | aboard the Titanic       |                                                |
| ticket    | Ticket number            |                                                |
| fare      | Passenger fare           |                                                |
| cabin     | Cabin number             |                                                |
| embarked  | Port of Embarkation      | C = Cherbourg, Q = Queenstown, S = Southampton |

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

from rapidfuzz import process, fuzz
from unidecode import unidecode

import warnings
warnings.filterwarnings("ignore")

train_df = pd.read_csv('./Data/titanic/train.csv')
test_df = pd.read_csv('./Data/titanic/test.csv')
df = pd.concat([train_df, test_df], ignore_index=True, sort=False)

df = df.convert_dtypes()

print(df.shape)
df.head()

(1309, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,<NA>,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,<NA>,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,<NA>,S


For quick error investigation, we can use the information printed above as well as the number of unique and missing values, and the data type of the column.

In [2]:
for col in df.columns:
    print(f"{col}\nUnique: {df[col].nunique()}\nMissing: {df[col].isna().sum()}\ndtype: {df[col].dtype}\n")

PassengerId
Unique: 1309
Missing: 0
dtype: Int64

Survived
Unique: 2
Missing: 418
dtype: Int64

Pclass
Unique: 3
Missing: 0
dtype: Int64

Name
Unique: 1307
Missing: 0
dtype: string

Sex
Unique: 2
Missing: 0
dtype: string

Age
Unique: 98
Missing: 263
dtype: Float64

SibSp
Unique: 7
Missing: 0
dtype: Int64

Parch
Unique: 8
Missing: 0
dtype: Int64

Ticket
Unique: 929
Missing: 0
dtype: string

Fare
Unique: 281
Missing: 1
dtype: Float64

Cabin
Unique: 186
Missing: 1014
dtype: string

Embarked
Unique: 3
Missing: 2
dtype: string



Let's tackle these columns individually. First up, `PassengerId` is good to go. It's an integer column with 1309 unique values, matching the dataset's length. 

Moving on to the next column, `Survived`. It aligns with the sample values, but there's a gap with 418 missing entries. To address those missing entries, we can consider filling them by scraping information from the web. One useful source is <a href="https://titanicfacts.net/titanic-passenger-list/">titanicfacts.net</a>, which presents passenger details such as age, embarkation, and survival in three well-organized tables.

In [3]:
r = requests.get("https://titanicfacts.net/titanic-passenger-list/")

soup = BeautifulSoup(r.text, 'lxml')
data_table = soup.find_all('table')

tables = pd.read_html(str(data_table), flavor="bs4", header=0)

# Concatenate tables
scraped_tfacts = pd.concat(tables, ignore_index=True)
scraped_tfacts = scraped_tfacts.convert_dtypes()

scraped_tfacts.head()

,Surname,First Names,Age,Boarded,Survivor (S) or Victim (†)
0,Allen,Miss Elisabeth Walton,29,Southampton,S
1,Allison,Mr Hudson Joshua Creighton,30,Southampton,†
2,Allison,Mrs Bessie Waldo,25,Southampton,†
3,Allison,Miss Helen Loraine,2,Southampton,†
4,Allison,Master Hudson Trevor,11m,Southampton,S


After scraping the data, it will require some cleaning to ensure its usability. Basically, we need to convert the data to a format consistent with our main df

In [4]:
# Format name as in df
scraped_tfacts["Name"] = scraped_tfacts["Surname"] + ", " + scraped_tfacts["First Names"]

# Mark unknown age as missing
scraped_tfacts["Age"] = scraped_tfacts["Age"].replace("NK", pd.NA)

# Convert months to fractions
months = scraped_tfacts["Age"].str.contains("[^\d]+", regex=True)
months_idx = scraped_tfacts.loc[months, "Age"].index

months_subset = scraped_tfacts.loc[scraped_tfacts["Age"].index.isin(months_idx)].copy()
months_subset["Age"] = months_subset["Age"].str.replace("m", "")
months_subset["Age"] = (months_subset["Age"].astype("Int64") / 12).round(2)

## Prevent TypeError raised from assigning floats to integer column
scraped_tfacts["Age"] = scraped_tfacts["Age"].str.replace("m", "").astype("Float64")
scraped_tfacts.loc[scraped_tfacts.index.isin(months_subset.index), "Age"] = months_subset["Age"]

scraped_tfacts["Embarked"] = scraped_tfacts["Boarded"].str[0]
scraped_tfacts["Survived"] = scraped_tfacts["Survivor (S) or Victim (†)"].replace({"S" : "1", "†" : "0"}).astype("Int64")

scraped_tfacts = scraped_tfacts.drop(["Surname","First Names","Boarded","Survivor (S) or Victim (†)"], axis=1)

print(scraped_tfacts.shape)
scraped_tfacts.head()

(1317, 4)


,Age,Name,Embarked,Survived
0,29.0,"Allen, Miss Elisabeth Walton",S,1
1,30.0,"Allison, Mr Hudson Joshua Creighton",S,0
2,25.0,"Allison, Mrs Bessie Waldo",S,0
3,2.0,"Allison, Miss Helen Loraine",S,0
4,0.92,"Allison, Master Hudson Trevor",S,1


Now that we have the data in a usable state, our next task is to find a way to incorporate the scraped survival values into our original dataset. The `Name` column can serve as a mediator for this purpose, but we must be cautious as small variations in names might hinder our ability to match individuals accurately between the two datasets.

To improve the reliability of the values in `Name`, we can use fuzzy string matching and assume that highly similar names across the two datasets represent the same individual. We can even take this a step further and cross-reference the `Embarked` column, which has only two missing entries, to validate the accuracy of our matched names.

Given that scraped data can also be used to fill in other columns like "Age," it's a good idea to encapsulate this logic into a function for reusability.

In [5]:
def fill_missing_values(df, scraped_df, column_to_fill):
    
    # Ensure name matching is not case-sensitive
    scraped_df["Name"] = scraped_df["Name"].str.lower()

    missing_rows = df.loc[df[column_to_fill].isna()]
    choices = scraped_df["Name"].tolist()

    for idx, row in missing_rows.iterrows():
        best_match = process.extract(row["Name"].lower(), choices, scorer=fuzz.WRatio)
        matched_name, similarity_score = best_match[0][0], best_match[0][1]

        # Validate with "Embarked" column 
        scraped_embarked = scraped_df.loc[scraped_df["Name"] == matched_name, "Embarked"].values[0]

        if similarity_score >= 85 and row["Embarked"] == scraped_embarked:
            # Get the corresponding value from scraped data
            scraped_value = scraped_df.loc[(scraped_df["Name"] == matched_name) & (scraped_df["Embarked"] == scraped_embarked), column_to_fill].values[0]

            # Update df
            df.at[idx, column_to_fill] = scraped_value

    return df

In [6]:
df = fill_missing_values(df, scraped_tfacts, "Survived")
print(f"Remaining missing: {df['Survived'].isna().sum()}")

Remaining missing: 71


We've managed to reduce the number of missing values in the `Survived` column from 418 down to just 71!

Now, let's shift our focus to the next column. `Name` has only 1307 unique values whereas the total number of rows is 1309. This suggests that some individuals may have duplicated records or share the same name.

In [7]:
df.loc[df["Name"].duplicated(keep=False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
289,290,1,3,"Connolly, Miss. Kate",female,22.0,0,0,370373,7.75,<NA>,Q
696,697,0,3,"Kelly, Mr. James",male,44.0,0,0,363592,8.05,<NA>,S
891,892,<NA>,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,<NA>,Q
897,898,0,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,<NA>,Q


Given the differences in data among these individuals, such as ticket numbers, ages, and other factors, we can confidently conclude that the records with shared names are indeed unique entries.

`Age` should also be investigated for two reasons:
1. You would expect age values to be integers, but they are floats.
2. Values outside the reasonable range could exist.

First, let's look for actual floats within the column.

In [8]:
# Locate instances with non-zero values after the decimal point
age_floats = df["Age"].astype("string").str.contains("\.[^0]+$")
df.loc[age_floats, "Age"].head()

57     28.5
78     0.83
111    14.5
116    70.5
122    32.5
Name: Age, dtype: Float64

Kaggle's description of the data mentions that the "age" column may contain fractional values if the age is less than 1, and estimated ages are represented in the form of xx.5. To validate this information, we'll need to gather names and a few examples.

In [9]:
df.loc[age_floats][["Name","Age"]]

,Name,Age
57,"Novel, Mr. Mansouer",28.5
78,"Caldwell, Master. Alden Gates",0.83
111,"Zabour, Miss. Hileni",14.5
116,"Connors, Mr. Patrick",70.5
122,"Nasser, Mr. Nicholas",32.5
123,"Webber, Miss. Susan",32.5
148,"Navratil, Mr. Michel (""Louis M Hoffman"")",36.5
152,"Meo, Mr. Alfonzo",55.5
153,"van Billiard, Mr. Austin Blyler",40.5
203,"Youseff, Mr. Gerious",45.5


Here is our random pick:

- Baclini, Miss. Helene Barbara (index 469): 0.75 years old
- Peacock, Master. Alfred Edward (index 1172): 0.75 years old
- Hanna, Mr. Mansour (index 296): 23.5 years old
- Mangan, Miss. Mary (index 767): 30.5 years old

Using <a href="www.encyclopedia-titanica.org">encyclopedia-titanica.org</a> we can determine how accurate this information is. According to the site and considering the Titanic sank in April 1912, the sample data should be:
- Baclini, Miss. Helene Barbara (<a href="https://www.encyclopedia-titanica.org/titanic-survivor/helene-baclini.html">source</a>): 9 months or 3 years old (age under dispute)
- Peacock, Master. Alfred Edward (<a href="https://www.encyclopedia-titanica.org/titanic-victim/albert-edward-peacock.html">source</a>): 7 months old
- Hanna, Mr. Mansour (<a href="https://www.encyclopedia-titanica.org/titanic-victim/mansour-hanna.html">source</a>): 35 years old
- Mangan, Miss. Mary (<a href="https://www.encyclopedia-titanica.org/titanic-victim/mary-mangan.html">source</a>): 32 years old


Our dataset contains some inaccuracies, but we have a solution at hand. By leveraging the scraped data we've collected and the function we've created to fill in missing values, we can significantly enhance the quality and completeness of the "Age" column.

We'll simplify the process by first replacing fractional and estimated age values with nulls. This paves the way for using our function to fill in all the missing age values in one step.

In [10]:
df.loc[age_floats, "Age"] = pd.NA
df = fill_missing_values(df, scraped_tfacts, "Age")

print(f"Remaining missing: {df['Age'].isna().sum()}")

Remaining missing: 80


We've made significant progress, reducing the number of missing age values from 263 to just 80 while improving data quality. As a final check for the "Age" column, let's determine the minimum and maximum values to ensure they fall within reasonable boundaries.

In [11]:
print(f"Min age: {df['Age'].min()}\nMax age: {df['Age'].max()}")

Min age: 0.17
Max age: 80.0


The minimum and maximum values seem reasonable. For example, the fractional value of 0.17 is roughly equivalent to two months old. A quick Google search confirms that Eliza Gladys Dean was the youngest passenger on the Titanic at just two months old. This further validates the accuracy of our data.

In [12]:
df.loc[df["Age"] == 0.17, "Name"].values[0]

'Dean, Miss. Elizabeth Gladys Millvina""'

Columns "SibSp" and "Parch" only need straightforward checks to verify that their values fall within a reasonable range. This is necessary because not all of their values were visible in the sample we obtained using `.head()` at the start of this analysis.

In [13]:
for col in ["SibSp","Parch"]:
    print(f"{col}: {df[col].unique()}\n")

SibSp: <IntegerArray>
[1, 0, 3, 4, 2, 5, 8]
Length: 7, dtype: Int64

Parch: <IntegerArray>
[0, 1, 2, 5, 3, 4, 6, 9]
Length: 8, dtype: Int64



We've arrived at the most challenging and intriguing column: "Cabin." This column exhibits significant variability for a categorical variable and presents the highest number of missing values. Therefore, our objective is twofold: 
1. Reduce cardinality.
2. Reduce the amount of missing data.

To address the high cardinality issue, one approach is to retain only the section information and omit the cabin number.

In [14]:
df["Cabin"] = df["Cabin"].str.replace("\d+", "", regex=True)
df["Cabin"].value_counts()

Cabin
C          80
B          48
D          44
E          40
A          22
F          14
B B         8
C C         8
C C C       6
B B B B     5
G           5
F G         4
B B B       4
F E         3
D D         2
T           1
E E         1
Name: count, dtype: Int64

The `.value_counts()` method provides us with counts for each unique value in the column. Interestingly, it seems that certain passengers have been assigned multiple values. Let's find out what this means.

In [15]:
df.loc[df["Cabin"].str.contains("C C C")]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C C C,S
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0,C C C,S
341,342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0,C C C,S
438,439,0,1,"Fortune, Mr. Mark",male,64.0,1,4,19950,263.0,C C C,S
944,945,1,1,"Fortune, Miss. Ethel Flora",female,28.0,3,2,19950,263.0,C C C,S
960,961,0,1,"Fortune, Mrs. Mark (Mary McDougald)",female,60.0,1,4,19950,263.0,C C C,S


These individuals share some remarkable similarities: they each have five relatives on board, share the same ticket number, paid an identical fare, and embarked from Southampton. Additionally, they all share the same surname. This compelling evidence strongly suggests that they were traveling together as a single family unit. Therefore, when multiple cabins are assigned, we can reasonably assume that it indicates a group of relatives traveling together.

This valuable information can be used to create new features that have the potential to enhance the predictive capabilities of any models we develop, including those for imputation. For the time being, let's keep the data as it is and shift our focus to locating the missing values for the cabin column.

To address the missing data issue, we'll need external sources. However, the scraped data we have previously collected isn't suitable for this purpose, as it lacks a "Cabin" column. Instead, we can utilize <a href="https://www.encyclopedia-titanica.org/cabins.html">encyclopedia-titanica.org</a> to obtain the necessary information for "Cabin".

In [16]:
r = requests.get("https://www.encyclopedia-titanica.org/cabins.html")

soup = BeautifulSoup(r.text, 'lxml')
data_table = soup.find('table')

scraped_ency_titanica = pd.read_html(str(data_table), flavor="bs4", header=0)[0]
scraped_ency_titanica = scraped_ency_titanica.convert_dtypes()

print(scraped_ency_titanica.shape)
scraped_ency_titanica

(251, 4)


,Cabin No.,Name,Evidence,Comments
0,Boat Deck,<NA>,<NA>,"6 cabins, accommodation for 7 people."
1,"""T""","Blackwell, Mr Stephen Weart",Cave List,There is no evidence that anyone else occupied...
2,<NA>,<NA>,<NA>,<NA>
3,A Deck,<NA>,<NA>,"36 cabins, accommodation for 48 people."
4,A-5,"Goldschmidt, Mr George B.",Cave List,<NA>
...,...,...,...,...
246,73 (?),"Moen, Mr Sigurd Hansen Søholt, Mr Peter Andrea...",Unknown,In section G
247,G Deck,<NA>,<NA>,<NA>
248,6 (?),"Sandström, Mrs Hjalmar Sandström, Miss Beatric...",Unknown,There is no cabin 6 in section G on F deck. Mo...
249,<NA>,"Ström, Mrs Wilhelm (Elna Matilda Persson) Strö...",Immigration Ticket,<NA>


The dataset contains 251 rows and four columns. We plan to get the information in "Cabin No." into our main dataset, just as we did before with "Survived" and "Age." To make this happen, we will also need the "Name" column. However, both "Cabin No." and "Name" contain missing entries so we will have to eliminate these. 

In [17]:
name_notna = scraped_ency_titanica["Name"].notna()
cabin_notna = scraped_ency_titanica["Cabin No."].notna()

scraped_ency_titanica = scraped_ency_titanica.loc[name_notna & cabin_notna]
scraped_ency_titanica

,Cabin No.,Name,Evidence,Comments
1,"""T""","Blackwell, Mr Stephen Weart",Cave List,There is no evidence that anyone else occupied...
4,A-5,"Goldschmidt, Mr George B.",Cave List,<NA>
5,A-6 or A-8 (?),"Beattie, Mr Thomson McCaffry, Mr Thomas Francis",Peuchen's testimony at the Senate inquiry,When Peuchen is asked about where his friends ...
6,A-7,"Smith, Mr James Clinch",Cave List,<NA>
7,A-9,"Chevré, Mr Paul Romaine",Cave List,<NA>
...,...,...,...,...
244,57 (?),"Krekorian, Mr Neshan",Unknown,In section E
245,63 (?),"Abelseth, Mr Olaus Jørgensen Humblen, Mr Adolf...",Unknown,In section G
246,73 (?),"Moen, Mr Sigurd Hansen Søholt, Mr Peter Andrea...",Unknown,In section G
248,6 (?),"Sandström, Mrs Hjalmar Sandström, Miss Beatric...",Unknown,There is no cabin 6 in section G on F deck. Mo...


Now that our data is somewhat more organized, let's delve into a couple of key observations. Firstly, within the "Name" column, there are instances where more than one passenger is bundled into a single value. If we can untangle these passengers into individual rows, it might augment the size of our dataset, potentially enhancing our capacity to fill in missing information for our primary dataset. Secondly, for cases where the cabin section is absent in "Cabin No.", we can extract this information from the "Comments" column.

For instance, consider row index 246. Here, passengers "Moen, Mr Sigurd HansenSøholt" and "Mr Peter Andreas" are distinct individuals paired into the same entry in the "Name" column. Interestingly, their cabin section is clearly indicated as G in the comments.

Our next move involves extracting the cabin section from the "Comments" column for cabin values lacking this detail. To preserve this newfound data, we'll introduce a new column, "Cabin_section". Subsequently, we can strategize on how to separate paired individuals into distinct rows.

In [18]:
scraped_ency_titanica["Cabin_section"] = scraped_ency_titanica["Comments"].str.extract("section ([A-Z])", expand=False)
scraped_ency_titanica

,Cabin No.,Name,Evidence,Comments,Cabin_section
1,"""T""","Blackwell, Mr Stephen Weart",Cave List,There is no evidence that anyone else occupied...,<NA>
4,A-5,"Goldschmidt, Mr George B.",Cave List,<NA>,<NA>
5,A-6 or A-8 (?),"Beattie, Mr Thomson McCaffry, Mr Thomas Francis",Peuchen's testimony at the Senate inquiry,When Peuchen is asked about where his friends ...,<NA>
6,A-7,"Smith, Mr James Clinch",Cave List,<NA>,<NA>
7,A-9,"Chevré, Mr Paul Romaine",Cave List,<NA>,<NA>
...,...,...,...,...,...
244,57 (?),"Krekorian, Mr Neshan",Unknown,In section E,E
245,63 (?),"Abelseth, Mr Olaus Jørgensen Humblen, Mr Adolf...",Unknown,In section G,G
246,73 (?),"Moen, Mr Sigurd Hansen Søholt, Mr Peter Andrea...",Unknown,In section G,G
248,6 (?),"Sandström, Mrs Hjalmar Sandström, Miss Beatric...",Unknown,There is no cabin 6 in section G on F deck. Mo...,G


After extracting the cabin section, we'll fill in the NAs with information from "Cabin No." and then clean up the column by removing any non-alphanumeric characters.

In [19]:
scraped_ency_titanica["Cabin_section"] = scraped_ency_titanica["Cabin_section"].fillna(scraped_ency_titanica["Cabin No."])
scraped_ency_titanica["Cabin_section"] = scraped_ency_titanica["Cabin_section"].str.replace("[^A-Z0-9]", "", regex=True)

To ensure data accuracy, we can check for values that contain more than one alphabetic character, as we've observed that cabin values typically adhere to this format.

In [20]:
inaccurate_cabins = scraped_ency_titanica["Cabin_section"].str.contains("[A-Z].*[A-Z]")
scraped_ency_titanica[["Cabin No.", "Cabin_section"]].loc[inaccurate_cabins]

,Cabin No.,Cabin_section
5,A-6 or A-8 (?),A6A8
188,E Deck (?E-10),EDE10
202,E-39 or E-41,E39E41
231,Third Class,TC
232,Unknown Deck,UD


What's next is to exclude the rows at indices 231 and 232 since they don't pertain to cabin sections. Then, we should eliminate any numeric values from the column and extract only the first letter, which signifies the cabin section. Finally, we should drop any null values created by removing numeric instances.

In [21]:
scraped_ency_titanica = scraped_ency_titanica.drop([231,232])
scraped_ency_titanica["Cabin_section"] = scraped_ency_titanica["Cabin_section"].str.replace("[0-9]", "", regex=True)
scraped_ency_titanica["Cabin_section"] = scraped_ency_titanica["Cabin_section"].str[0]

scraped_ency_titanica = scraped_ency_titanica.dropna(subset=['Cabin_section'])

scraped_ency_titanica["Cabin_section"].value_counts()

Cabin_section
C    66
B    46
E    35
D    33
A    21
G     3
F     3
M     2
T     1
O     1
Q     1
Name: count, dtype: Int64

Now, let's dive into the more exciting task of separating paired passengers and assigning each to their dedicated row. There are NLP tools for recognizing name entities in text, but these can be time-consuming and difficult to set up. Instead, we can develop our own algorithm using the information at hand.

In [22]:
scraped_ency_titanica["Name"].sample(20, random_state=1).to_numpy()

array(['Stähelin, Dr Max', 'Gee, Mr Arthur H.',
       'Davidson, Mr Thornton Davidson, Mrs Thornton (Orian Hays)',
       'Thayer, Mr John Borland, jr.', 'Stewart, Mr Albert A.',
       'Straus, Mr Isidor Straus, Mrs Isidor (Ida Blun)',
       'Stone, Mrs George Nelson Icard, Miss Amelie ("Amelia _") [maid to Mrs Stone]',
       'Beckwith, Mr Richard Leonard Beckwith, Mrs Richard Leonard (Sallie Monypeny)',
       'Gilnagh, Miss Katherine "Katie" Mullen, Miss Katherine "Katie" ("_Mullins") Murphy, Miss Margaret Jane ("Mary") Miss Katherine "Kate"',
       'Tobin, Mr Roger',
       'Mardirosian, Mr Sarkis (+ three other steerage)',
       'Chevré, Mr Paul Romaine',
       'Ryerson, Mr Arthur Larned Ryerson, Mrs Arthur Larned',
       'Rood, Mr Hugh R.', 'Lurette, Miss Elise [maid to Mrs Spencer]',
       'Sage, Master William Henry (and possibly others from the Sage family)',
       'Francatelli, Miss Laura Mabel [maid to Lady Duff Gordon]',
       'Pears, Mr Thomas Pears, Mrs Thomas (

From the generated sample, we observe that individual names are formatted as: <i>name, title name</i>

This is evident in values that contain one individual name like in "Stähelin, Dr Max" and "Gee, Mr Arthur H.". However, the formatting is also the same for values containing multiple individuals. For example, in "Davidson, Mr Thornton Davidson, Mrs Thornton (Orian Hays)", the people involved are "Davidson, Mr Thornton" and "Davidson, Mrs Thornton (Orian Hays)". This can be validated with a simple google search which I've done with several different examples.

Understanding how names are structured allows us to identify patterns for recognizing unique entities. One potential solution is to use the person's title to discover how many individuals exist within a particular string. Notice that the title always comes after a comma and a space, followed by another space. Knowing this, we can create a regex that matches each entity.

Starting with title, the equivalent regex would be: `",\s[A-Za-z]+\s"` (matching: ", Mr "). We should add to this expression so that it also matches names on the left of the title: `"[A-Za-z]+,\s[A-Za-z]+\s"` (matching: "Davidson, Mr "). Finally, to match names coming after the title only if they are not immediately followed by a comma - as this would indicate the beginning of another entity - can be achieved by adding a non-capturing group that looks for text separated by a space, one or more times: `"[A-Za-z]+,\s[A-Za-z]+\s(?:[A-Za-z]+\s)+"` (matching: "Davidson, Mr Thornton ").

We now have a regex that works in theory, but not in practise. There are several issues in the data that must be eliminated first, including:
- non-ASCII characters: 
- Junk information presented in square brackets
- Irrelevant punctuation

In addition, there are examples including information about the entity in parenthesis which is useful (e.g. "Davidson, Mrs Thornton (Orian Hays)"). We would like to keep this information while getting rid of the parenthesis.

In [23]:
scraped_ency_titanica["Name"] = scraped_ency_titanica["Name"].apply(unidecode)
scraped_ency_titanica["Name"] = scraped_ency_titanica["Name"].str.replace("\[.+\]", "", regex=True)
scraped_ency_titanica["Name"] = scraped_ency_titanica["Name"].str.replace("[^A-Za-z,\s]", "", regex=True)
scraped_ency_titanica["Name"] = scraped_ency_titanica["Name"].str.replace("\(\)", "", regex=True)
scraped_ency_titanica["Name"] = scraped_ency_titanica["Name"].str.strip()

scraped_ency_titanica["Name"].sample(20, random_state=1).to_numpy()

array(['Stahelin, Dr Max', 'Gee, Mr Arthur H',
       'Davidson, Mr Thornton Davidson, Mrs Thornton Orian Hays',
       'Thayer, Mr John Borland, jr', 'Stewart, Mr Albert A',
       'Straus, Mr Isidor Straus, Mrs Isidor Ida Blun',
       'Stone, Mrs George Nelson Icard, Miss Amelie Amelia',
       'Beckwith, Mr Richard Leonard Beckwith, Mrs Richard Leonard Sallie Monypeny',
       'Gilnagh, Miss Katherine Katie Mullen, Miss Katherine Katie Mullins Murphy, Miss Margaret Jane Mary Miss Katherine Kate',
       'Tobin, Mr Roger', 'Mardirosian, Mr Sarkis  three other steerage',
       'Chevre, Mr Paul Romaine',
       'Ryerson, Mr Arthur Larned Ryerson, Mrs Arthur Larned',
       'Rood, Mr Hugh R', 'Lurette, Miss Elise',
       'Sage, Master William Henry and possibly others from the Sage family',
       'Francatelli, Miss Laura Mabel',
       'Pears, Mr Thomas Pears, Mrs Thomas Edith Wearne',
       'Kent, Mr Edward Austin', 'Ostby, Mr Engelhart Cornelius'],
      dtype=object)

This makes the data compatible with our regex. 

In [24]:
# wrap regex in parenthesis for str.extract
scraped_entities = scraped_ency_titanica["Name"].str.extractall("([A-Za-z]+,\s[A-Za-z]+\s(?:[A-Za-z]+\s)+)").unstack()[0]
scraped_entities.head()

match,0,1,2,3
1,"Blackwell, Mr Stephen",NaN,NaN,NaN
4,"Goldschmidt, Mr George",NaN,NaN,NaN
5,"Beattie, Mr Thomson","McCaffry, Mr Thomas",NaN,NaN
6,"Smith, Mr James",NaN,NaN,NaN
7,"Chevre, Mr Paul",NaN,NaN,NaN


In [25]:
scraped_entities= pd.melt(scraped_entities, ignore_index=False).dropna().drop("match", axis=1)
scraped_entities["Cabin"] = scraped_ency_titanica["Cabin_section"]
scraped_entities.head()

,value,Cabin
1,"Blackwell, Mr Stephen",T
4,"Goldschmidt, Mr George",A
5,"Beattie, Mr Thomson",A
6,"Smith, Mr James",A
7,"Chevre, Mr Paul",A


In [26]:
def fill_missing_cabin(df, scraped_df, column_to_fill):
    
    # Ensure name matching is not case-sensitive
    scraped_df["value"] = scraped_df["value"].str.lower()

    missing_rows = df.loc[df[column_to_fill].isna()]
    choices = scraped_df["value"].tolist()

    for idx, row in missing_rows.iterrows():
        best_match = process.extract(row["Name"].lower(), choices, scorer=fuzz.WRatio)
        matched_name, similarity_score = best_match[0][0], best_match[0][1]

        if similarity_score >= 90:
            # Get the corresponding value from scraped data
            scraped_value = scraped_df.loc[scraped_df["value"] == matched_name, column_to_fill].values[0]

            # Update df
            df.at[idx, column_to_fill] = scraped_value

    return df

In [27]:
new_df = fill_missing_cabin(df, scraped_entities, "Cabin")
new_df["Cabin"].isna().sum()

1009